# Sparsity analysis

In [ ]:
import torch
from collections import deque
import os

import networkx as nx
import dgl

import matplotlib.pyplot as plt
import numpy as np

import polygraphs as pg
from polygraphs import graphs
from polygraphs import hyperparameters as hparams
from polygraphs import visualisations as viz
from polygraphs import ops

%matplotlib inline

## Static analysis

### A. Measures

In [ ]:
params = hparams.NetworkHyperParameters()
params.kind = 'wattsstrogatz'
params.size = 32
params.wattsstrogatz.knn = 4
params.wattsstrogatz.probability = 0.5
params.selfloop = False

# Calling graph constructor indirectly
graph = graphs.create(params)
_ = viz.draw(graph, figsize=(10, 8), layout='circular')

#### Sparsity

In [ ]:
def sparsity(graph):
    """
    Returns sparsity level of given DGL graph.
    """
    # Assumes an adjacency matrix of size N x N with M non-zero values
    return graph.num_edges() / (graph.num_nodes() ** 2)


sparsity(graph)

#### Clustering coefficient

In [ ]:
def acc(graph):
    """
    Returns average clustering coefficient.
    """
    graphx = nx.DiGraph(dgl.to_networkx(graph))
    return nx.algorithms.cluster.average_clustering(graphx)


acc(graph)

#### Average path length

In [ ]:
def apl(graph):
    """
    Returns average shortest path length.
    """
    graphx = nx.DiGraph(dgl.to_networkx(graph))
    return nx.average_shortest_path_length(graphx)


apl(graph)

## Dynamic analysis

In [ ]:
# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.001

params.network.kind = 'complete'
params.network.size = 1024

# Enable logging; print progress every 100 steps
params.logging.enabled = True
params.logging.interval = 100

# Take snapshots
params.snapshots.enabled = True
params.snapshots.interval = 100

params.simulation.steps = 0
params.simulation.repeats = 1

# Store results in directory
params.simulation.results = "data/1024-complete"

# Set seed
params.seed = 123456789

pg.random(params.seed)
_ = pg.simulate(params, op=ops.BalaGoyalOp)

In [ ]:
import h5py


def sparsity(graph):
    """
    Returns sparsity level of given DGL graph.
    """
    # Remove self-loops
    g = dgl.remove_self_loop(graph)
    # Assumes an adjacency matrix of size N x N with M non-zero values
    return g.num_edges() / (g.num_nodes() ** 2)


def acc(graph):
    """
    Returns average clustering coefficient.
    """
    graphx = nx.DiGraph(dgl.to_networkx(graph))
    return nx.algorithms.cluster.average_clustering(graphx)


def apl(graph):
    """
    Returns average shortest path length.
    """
    graphx = nx.DiGraph(dgl.to_networkx(graph))
    return nx.average_shortest_path_length(graphx)


def filterfn(edges):
    return torch.le(edges.src["beliefs"], 0.5)
   

def postprocess(directory, id):
    """
    Post-process graph snapshots
    """
    # Resulting hashtable
    ht = {}
    graphs, _ = dgl.load_graphs(os.path.join(directory, f"{id}.bin"))
    graph = graphs[0]
    fp = h5py.File(os.path.join(directory, f"{id}.hd5"), "r")
    _keys = [int(key) for key in fp.keys()]
    _keys = sorted(_keys)
    for key in _keys:
        graph.ndata["beliefs"] = torch.tensor(fp[str(key)][:])
        # Filter any edge whose source has belief less than 0.5
        inactive = graph.filter_edges(filterfn)
        # Create subgraph
        subgraph = dgl.remove_edges(graph, inactive)
        # Debugging
        s = 'DBG> '
        s += f'From G({graph.num_nodes():4d}, {graph.num_edges():5d})'
        s += f'to G\'({subgraph.num_nodes():4d}, {subgraph.num_edges():5d})'
        print(s)
        # Compute network statistics
        ht[key] = sparsity(subgraph)
    return ht


ht = postprocess("data/128-complete", 1)

In [ ]:
plt.rc('font', **{'family':'sans-serif','sans-serif':['Arial'], 'size': 16})

# Configure the y-axis - find max value and use discrete steps
plt.ylim([0, 1])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.ylabel('Sparsity')
# Configure the x-axis
plt.xlim([0, max(ht.keys())])
plt.xlabel('Iterations')
# Create a bar chart
plt.plot(ht.keys(), 
         ht.values(),
         '--o',
         color='#B7BF35')